### PART 1. Data preprocessing and visualization

In [3]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

2024-04-10 23:20:49.103682: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-10 23:20:49.147750: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-10 23:20:49.147792: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-10 23:20:49.150476: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-10 23:20:49.161406: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-10 23:20:49.163229: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [4]:
import shutil
try:
    shutil.rmtree("data_all_modified/train")
    shutil.rmtree("data_all_modified/test")
except:
    pass

In [5]:
import os

# ensure directories exist
from pathlib import Path
Path("data_all_modified-split/train/damage").mkdir(parents=True, exist_ok=True)
Path("data_all_modified-split/train/no_damage").mkdir(parents=True, exist_ok=True)

Path("data_all_modified-split/test/damage").mkdir(parents=True, exist_ok=True)
Path("data_all_modified-split/test/no_damage").mkdir(parents=True, exist_ok=True)

In [6]:
# we need paths of images for individual classes so we can copy them in the new directories that we created above
all_damage_file_paths = os.listdir('data_all_modified/damage')
all_no_damage_file_paths = os.listdir('data_all_modified/no_damage')

In [7]:
import random

train_damage_paths = random.sample(all_damage_file_paths, int(len(all_damage_file_paths)*0.8))
print("train damage image count: ", len(train_damage_paths))
test_damage_paths = [ p for p in all_damage_file_paths if p not in train_damage_paths]
print("test damage image count: ", len(test_damage_paths))
# ensure no overlap:
overlap = [p for p in train_damage_paths if p in test_damage_paths]
print("len of overlap: ", len(overlap))

train_no_damage_paths = random.sample(all_no_damage_file_paths, int(len(all_no_damage_file_paths)*0.8))
print("train no damage image count: ", len(train_no_damage_paths))
test_no_damage_paths = [ p for p in all_no_damage_file_paths if p not in train_no_damage_paths]
print("test no damage image count: ", len(test_no_damage_paths))
# ensure no overlap:
overlap = [p for p in train_no_damage_paths if p in test_no_damage_paths]
print("len of overlap: ", len(overlap))

train damage image count:  11336
test damage image count:  2834
len of overlap:  0
train no damage image count:  5721
test no damage image count:  1431
len of overlap:  0


In [8]:
# ensure to copy the images to the directories
import shutil
for p in train_damage_paths:
    shutil.copyfile(os.path.join('data_all_modified/damage', p), os.path.join('data_all_modified-split/train/damage', p) )

for p in test_damage_paths:
    shutil.copyfile(os.path.join('data_all_modified/damage', p), os.path.join('data_all_modified-split/test/damage', p) )

for p in train_no_damage_paths:
    shutil.copyfile(os.path.join('data_all_modified/no_damage', p), os.path.join('data_all_modified-split/train/no_damage', p) )

for p in test_no_damage_paths:
    shutil.copyfile(os.path.join('data_all_modified/no_damage', p), os.path.join('data_all_modified-split/test/no_damage', p) )

# check counts:
print("Files in train/damage: ", len(os.listdir("data_all_modified-split/train/damage")))
print("Files in train/no_damage: ", len(os.listdir("data_all_modified-split/train/no_damage")))

print("Files in test/damage: ", len(os.listdir("data_all_modified-split/train/damage")))
print("Files in test/no_damage: ", len(os.listdir("data_all_modified-split/train/no_damage")))

Files in train/damage:  14044
Files in train/no_damage:  7101
Files in test/damage:  14044
Files in test/no_damage:  7101


In [9]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

train_data_dir = 'data_all_modified-split/train'

batch_size = 32
# target image size
img_height = 128
img_width = 128

# note that subset="training", "validation", "both", and dictates which dataset is returned
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
train_data_dir,
validation_split=0.2,
subset="both",
seed=123,
image_size=(img_height, img_width),
batch_size=batch_size
)
rescale = Rescaling(scale=1.0/255)
train_rescale_ds = train_ds.map(lambda image,label:(rescale(image),label))
val_rescale_ds = val_ds.map(lambda image,label:(rescale(image),label))

Found 21145 files belonging to 2 classes.
Using 16916 files for training.
Using 4229 files for validation.


In [10]:
test_data_dir = 'data_all_modified-split/test'

batch_size = 2
# target image size
img_height = 128
img_width = 128

# note that subset="training", "validation", "both", and dictates which dataset is returned
test_ds = tf.keras.utils.image_dataset_from_directory(
test_data_dir,
seed=123,
image_size=(img_height, img_width),
)
rescale = Rescaling(scale=1.0/255)
test_rescale_ds = test_ds.map(lambda image,label:(rescale(image),label))

Found 10389 files belonging to 2 classes.


### PART 2. Model design, training, and evaluation. Exploring different model architectures, and identifying the model with high accuracy and validation accuracy.  

### Building A Dense ANN

In [24]:
from keras import layers
from keras import models
import pandas as pd
from keras import optimizers

# Intializing a sequential model
model_cnn = models.Sequential()

# Adding first conv layer with 64 filters and kernel size 3x3 , padding 'same' provides the output size same as the input size
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu', padding="same", input_shape=(128,128,3)))

# Adding max pooling to reduce the size of output of first conv layer
model_cnn.add(layers.MaxPooling2D((2, 2), padding = 'same'))

model_cnn.add(layers.Conv2D(32, (3, 3), activation='relu', padding="same"))
model_cnn.add(layers.MaxPooling2D((2, 2), padding = 'same'))

model_cnn.add(layers.Conv2D(32, (3, 3), activation='relu', padding="same"))
model_cnn.add(layers.MaxPooling2D((2, 2), padding = 'same'))

# flattening the output of the conv layer after max pooling to make it ready for creating dense connections
model_cnn.add(layers.Flatten())

# Adding a fully connected dense layer with 100 neurons
model_cnn.add(layers.Dense(100, activation='relu'))

# Adding a fully connected dense layer with 84 neurons
model_cnn.add(layers.Dense(84, activation='relu'))

# Adding the output layer with * neurons and activation functions as softmax since this is a multi-class classification problem
model_cnn.add(layers.Dense(1, activation='sigmoid'))

# Compile model
# RMSprop (Root Mean Square Propagation) is commonly used in training deep neural networks.
model_cnn.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
model_cnn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 128, 128, 64)      1792      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 64, 64, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 64, 64, 32)        18464     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 32, 32, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 16, 16, 32)       

In [25]:
#fit the model from image generator
history = model_cnn.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
529/529 [==============================] - 148s 278ms/step - loss: 0.5825 - accuracy: 0.7135 - val_loss: 0.4529 - val_accuracy: 0.8243
Epoch 2/20
529/529 [==============================] - 148s 280ms/step - loss: 0.4427 - accuracy: 0.8079 - val_loss: 0.4102 - val_accuracy: 0.8446
Epoch 3/20
529/529 [==============================] - 148s 279ms/step - loss: 0.3855 - accuracy: 0.8378 - val_loss: 0.3857 - val_accuracy: 0.8312
Epoch 4/20
529/529 [==============================] - 146s 275ms/step - loss: 0.3203 - accuracy: 0.8695 - val_loss: 0.3689 - val_accuracy: 0.8449
Epoch 5/20
529/529 [==============================] - 148s 280ms/step - loss: 0.2533 - accuracy: 0.9010 - val_loss: 0.2098 - val_accuracy: 0.9236
Epoch 6/20
529/529 [==============================] - 143s 271ms/step - loss: 0.1972 - accuracy: 0.9239 - val_loss: 0.2066 - val_accuracy: 0.9177
Epoch 7/20
529/529 [==============================] - 142s 268ms/step - loss: 0.1661 - accuracy: 0.9359 - val_loss: 0.1788 -

In [39]:
test_loss, test_accuracy = model_cnn.evaluate(test_rescale_ds, verbose=1)
test_accuracy

325/325 [==============================] - 24s 72ms/step - loss: 0.0583 - accuracy: 0.9786


0.9786312580108643

In [38]:
model_cnn.save("hurricane_cnn.keras")

### The Lenet-5 CNN architecture

In [30]:
from keras import layers
from keras import models
import pandas as pd
from tensorflow.keras import optimizers
from tensorflow.keras import layers, models, optimizers
model_lenet5 = models.Sequential()

# Layer 1: Convolutional layer with 6 filters of size 3x3, followed by average pooling
model_lenet5.add(layers.Conv2D(6, kernel_size=(3, 3), activation='relu', input_shape=(128,128,3)))
model_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Layer 2: Convolutional layer with 16 filters of size 3x3, followed by average pooling
model_lenet5.add(layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))
model_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Flatten the feature maps to feed into fully connected layers
model_lenet5.add(layers.Flatten())

# Layer 3: Fully connected layer with 120 neurons
model_lenet5.add(layers.Dense(120, activation='relu'))

# Layer 4: Fully connected layer with 84 neurons
model_lenet5.add(layers.Dense(84, activation='relu'))

# Output layer: Fully connected layer with num_classes neurons (e.g., 3 )
model_lenet5.add(layers.Dense(1, activation='sigmoid'))

# Compile model
model_lenet5.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
model_lenet5.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 126, 126, 6)       168       
                                                                 
 average_pooling2d_2 (Avera  (None, 63, 63, 6)         0         
 gePooling2D)                                                    
                                                                 
 conv2d_13 (Conv2D)          (None, 61, 61, 16)        880       
                                                                 
 average_pooling2d_3 (Avera  (None, 30, 30, 16)        0         
 gePooling2D)                                                    
                                                                 
 flatten_4 (Flatten)         (None, 14400)             0         
                                                                 
 dense_11 (Dense)            (None, 120)              

In [31]:
#fit the model from image generator
history = model_lenet5.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
529/529 [==============================] - 30s 56ms/step - loss: 0.6185 - accuracy: 0.6810 - val_loss: 0.6344 - val_accuracy: 0.6529
Epoch 2/20
529/529 [==============================] - 30s 57ms/step - loss: 0.5363 - accuracy: 0.7471 - val_loss: 0.5294 - val_accuracy: 0.7151
Epoch 3/20
529/529 [==============================] - 30s 56ms/step - loss: 0.4574 - accuracy: 0.8061 - val_loss: 0.4790 - val_accuracy: 0.7593
Epoch 4/20
529/529 [==============================] - 30s 56ms/step - loss: 0.4099 - accuracy: 0.8368 - val_loss: 0.4281 - val_accuracy: 0.8044
Epoch 5/20
529/529 [==============================] - 30s 56ms/step - loss: 0.3828 - accuracy: 0.8496 - val_loss: 0.3545 - val_accuracy: 0.8697
Epoch 6/20
529/529 [==============================] - 29s 55ms/step - loss: 0.3626 - accuracy: 0.8587 - val_loss: 0.6147 - val_accuracy: 0.7354
Epoch 7/20
529/529 [==============================] - 29s 56ms/step - loss: 0.3480 - accuracy: 0.8665 - val_loss: 0.3247 - val_accuracy:

In [32]:
test_loss, test_accuracy = model_lenet5.evaluate(test_rescale_ds, verbose=0)
test_accuracy

0.936663806438446

In [33]:
model_lenet5.save("hurricane_lenet.keras")

### Alternate-Lenet-5 CNN Architecture

In [34]:
model_Altlenet5 = models.Sequential()
# Layer 1: Convolutional layer with 6 filters of size 3x3, followed by average pooling
model_Altlenet5.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128,128,3)))
model_Altlenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Layer 2: Convolutional layer with 16 filters of size 3x3, followed by average pooling
model_Altlenet5.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model_Altlenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Layer 3: Convolutional layer with 16 filters of size 3x3, followed by average pooling
model_Altlenet5.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
model_Altlenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Layer 4: Convolutional layer with 16 filters of size 3x3, followed by average pooling
model_Altlenet5.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
model_Altlenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Flatten the feature maps to feed into fully connected layers
model_Altlenet5.add(layers.Flatten())

# Layer 3: Fully connected layer with 32 neurons
model_Altlenet5.add(layers.Dense(512, activation='relu'))

# Output layer: Fully connected layer with num_classes neurons (e.g., 3 )
model_Altlenet5.add(layers.Dense(1, activation='sigmoid'))

# Compile model
model_Altlenet5.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
model_Altlenet5.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 63, 63, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_15 (Conv2D)          (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 30, 30, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_16 (Conv2D)          (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 14, 14, 128)      

In [35]:
history = model_Altlenet5.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
529/529 [==============================] - 114s 213ms/step - loss: 0.5166 - accuracy: 0.7570 - val_loss: 0.4172 - val_accuracy: 0.8345
Epoch 2/20
529/529 [==============================] - 112s 211ms/step - loss: 0.3629 - accuracy: 0.8459 - val_loss: 0.3079 - val_accuracy: 0.8749
Epoch 3/20
529/529 [==============================] - 109s 206ms/step - loss: 0.2828 - accuracy: 0.8854 - val_loss: 0.2846 - val_accuracy: 0.8827
Epoch 4/20
529/529 [==============================] - 112s 211ms/step - loss: 0.2186 - accuracy: 0.9135 - val_loss: 0.2131 - val_accuracy: 0.9168
Epoch 5/20
529/529 [==============================] - 111s 209ms/step - loss: 0.1812 - accuracy: 0.9279 - val_loss: 0.1918 - val_accuracy: 0.9255
Epoch 6/20
529/529 [==============================] - 111s 210ms/step - loss: 0.1515 - accuracy: 0.9402 - val_loss: 0.1397 - val_accuracy: 0.9461
Epoch 7/20
529/529 [==============================] - 110s 209ms/step - loss: 0.1299 - accuracy: 0.9484 - val_loss: 0.2169 -

In [36]:
test_loss, test_accuracy = model_Altlenet5.evaluate(test_rescale_ds, verbose=0)
test_accuracy

0.9802675843238831

In [1]:
model_Altlenet5.save("hurricane_alt.keras")

NameError: name 'model_Altlenet5' is not defined